In [1]:

# importing packages 

import pandas as pd 
import datetime
import folium
from folium.plugins import MarkerCluster
from ipywidgets import interact


In [2]:
# importing data 

data_path = '/Users/adamisaiahhansen/Downloads/projects/Michigan_Salmonid/fish_project_data/'

steelhead_ = pd.read_csv(data_path  + 'Steelhead.csv')

atlantic_sal_ = pd.read_csv(data_path + 'Atlantic_salmon.csv')

chinook_sal_ = pd.read_csv(data_path + 'Chinook_salmon.csv')

coho_sal_ = pd.read_csv(data_path + 'Coho_salmon.csv')

lake_trout_ = pd.read_csv(data_path + 'Lake_trout.csv')

location = pd.read_csv(data_path + 'MI_County_Lat_Lng.csv')


In [3]:

# couting na values 

steelhead_.isna().sum()

atlantic_sal_.isna().sum()

chinook_sal_.isna().sum()

coho_sal_.isna().sum()

lake_trout_.isna().sum()


County                       0
Water Body                1577
Site Name                    0
Town                      1577
Range                     1577
Section                   1577
Species                      0
Strain                     608
Date                         0
Number                       0
Avg. Length                  0
Operation                    0
Fin Clips, Marks, Tags       0
dtype: int64

In [4]:

# cleaning the data 

# updating two counties for location 

location.replace("St. Clair", "Saint Clair", inplace=True) 
location.replace("St. Joseph", "Saint Joseph", inplace=True) 

# dropping not needed colums 

location = location.drop(['County_full', 'State_id', 'State_name'], axis=1)

# the below colums are currently not being used and contain all the na values 
steelhead = steelhead_.drop(['Water Body', 'Range','Town', 'Section', 'Strain'], axis=1)
atlantic_sal = atlantic_sal_.drop(['Water Body', 'Range','Town', 'Section', 'Strain'], axis=1)
chinook_sal = chinook_sal_.drop(['Water Body', 'Range','Town', 'Section', 'Strain'], axis=1)
coho_sal = coho_sal_.drop(['Water Body', 'Range','Town', 'Section', 'Strain'], axis=1)
lake_trout = lake_trout_.drop(['Water Body', 'Range','Town', 'Section', 'Strain'], axis=1)


steelhead.isna().sum()

atlantic_sal.isna().sum()

chinook_sal.isna().sum()

coho_sal.isna().sum()

lake_trout.isna().sum()


County                    0
Site Name                 0
Species                   0
Date                      0
Number                    0
Avg. Length               0
Operation                 0
Fin Clips, Marks, Tags    0
dtype: int64

In [5]:
%%capture --no-display
## stops the display of pandas warning, does not effect output

# seperating date to year and month

steelhead['Date_ydm'] = pd.to_datetime(steelhead['Date'])
steelhead['Year'], steelhead['Month'] = steelhead['Date_ydm'].dt.year, steelhead['Date_ydm'].dt.month

atlantic_sal['Date_ydm'] = pd.to_datetime(atlantic_sal['Date'])
atlantic_sal['Year'], atlantic_sal['Month'] = atlantic_sal['Date_ydm'].dt.year, atlantic_sal['Date_ydm'].dt.month

chinook_sal['Date_ydm'] = pd.to_datetime(chinook_sal['Date'])
chinook_sal['Year'], chinook_sal['Month'] = chinook_sal['Date_ydm'].dt.year,chinook_sal['Date_ydm'].dt.month

coho_sal['Date_ydm'] = pd.to_datetime(coho_sal['Date'])
coho_sal['Year'], coho_sal['Month'] = coho_sal['Date_ydm'].dt.year, coho_sal['Date_ydm'].dt.month

lake_trout['Date_ydm'] = pd.to_datetime(lake_trout['Date'])
lake_trout['Year'], lake_trout['Month'] = lake_trout['Date_ydm'].dt.year, lake_trout['Date_ydm'].dt.month


In [6]:

# joining location to fish stocking data 

steelhead = pd.merge(steelhead, location, on='County', how='left')
atlantic_sal = pd.merge(atlantic_sal, location, on='County', how='left')
chinook_sal = pd.merge(chinook_sal, location, on='County', how='left')
coho_sal = pd.merge(coho_sal, location, on='County', how='left')
lake_trout = pd.merge(lake_trout, location, on='County', how='left')


In [7]:

# one big data frame of all fish data 

frames = [steelhead, atlantic_sal, chinook_sal, coho_sal, lake_trout]

all_fish = pd.concat(frames)



In [8]:
#df[(df[Gender]=='Male') & (df[Year]==2014)]
all_fish.query('County=="Iosco" & Year==2020')

#all_fish.info()

,County,Site Name,Species,Date,Number,Avg. Length,Operation,"Fin Clips, Marks, Tags",Date_ydm,Year,Month,Lat,Ing
4595,Iosco,REA ROAD,Rainbow trout,3/25/2020 12:00:00 AM,37709,7.40,State Plant,"adipose, coded wire tag",2020-03-25,2020,3,44.3558,-83.6359
4596,Iosco,REA ROAD,Rainbow trout,3/31/2020 12:00:00 AM,26000,7.99,State Plant,"adipose, coded wire tag",2020-03-31,2020,3,44.3558,-83.6359
4597,Iosco,REA ROAD,Rainbow trout,4/1/2020 12:00:00 AM,12670,8.11,State Plant,"adipose, coded wire tag",2020-04-01,2020,4,44.3558,-83.6359
4598,Iosco,REA ROAD,Rainbow trout,4/10/2020 12:00:00 AM,25569,7.52,State Plant,"adipose, coded wire tag",2020-04-10,2020,4,44.3558,-83.6359
4599,Iosco,REA ROAD,Rainbow trout,4/10/2020 12:00:00 AM,3701,7.52,State Plant,adipose clip,2020-04-10,2020,4,44.3558,-83.6359
4632,Iosco,CARPENTER RD.,Rainbow trout,4/2/2020 12:00:00 AM,12100,7.32,State Plant,adipose clip,2020-04-02,2020,4,44.3558,-83.6359
4633,Iosco,CARPENTER RD.,Rainbow trout,4/2/2020 12:00:00 AM,24716,7.52,State Plant,"adipose, coded wire tag",2020-04-02,2020,4,44.3558,-83.6359
4634,Iosco,CARPENTER RD.,Rainbow trout,4/6/2020 12:00:00 AM,12400,8.03,State Plant,"adipose, coded wire tag",2020-04-06,2020,4,44.3558,-83.6359
281,Iosco,WHIRLPOOL PAS,Atlantic salmon,5/13/2020 12:00:00 AM,14560,6.62,State Plant,none,2020-05-13,2020,5,44.3558,-83.6359
282,Iosco,WHIRLPOOL PAS,Atlantic salmon,5/13/2020 12:00:00 AM,25440,6.53,State Plant,none,2020-05-13,2020,5,44.3558,-83.6359


In [9]:

# map of michigan 

mi_map = folium.Map(
    location = [44.3148, -85.6024], 
    zoom_start = 6)

mi_map    


In [10]:

def fish_map_all(fish_type, year): 
    
    fish = all_fish[all_fish["Year"] == year] 
    fish = fish[fish["Species"] == fish_type]
    fish = fish.groupby(['County','Lat','Ing'], as_index=False).agg(Stocked=('Number','sum'), 
                                            AvgLength= ('Avg. Length','mean'))
                                           
    fish.AvgLength = fish.AvgLength.round(decimals = 3)
    
    mi_map = folium.Map(
    location = [44.3148, -85.6024], 
    zoom_start = 6)
    
    for _, data in fish.iterrows(): 
        text = f"County: {data['County']} <br> Stocked: {data['Stocked']} <br> Avg Length (in): {data['AvgLength']}"
        folium.Marker(
        location = [data['Lat'], data['Ing']],
        #popup = data['County'] + ' ' + str(data['Stocked']),  
        tooltip = text).add_to(mi_map)
    
    return mi_map


In [11]:

unique_fish = all_fish.Species.unique()

map_fish = interact(fish_map_all, fish_type = unique_fish, year = (all_fish.Year.min(),all_fish.Year.max()))


map_fish 


interactive(children=(Dropdown(description='fish_type', options=('Rainbow trout', 'Atlantic salmon', 'Chinook …

<function __main__.fish_map_all(fish_type, year)>